<a href="https://colab.research.google.com/github/ritaalamino/candc_master/blob/main/CANDC_learnbyreading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## *CANDC*

Funções não suportadas

*   --semantics
*   --modal
*   --elimeq não muda nada
*   --vpe  não muda nada
*   --robust  não muda nada
*   --roles  não muda nada

##Importa bibliotecas

In [1]:
# Import das bibliotecas.

# Biblioteca de logging
import logging
import subprocess
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instala as libraries necessárias

In [3]:
# Diretório do cohebert
DIRETORIO_CANDC = "CANDC"

# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_CANDC + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_CANDC + "/"

In [4]:
!git clone https://github.com/valeriobasile/learningbyreading.git

Cloning into 'learningbyreading'...
remote: Enumerating objects: 1870, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 1870 (delta 30), reused 58 (delta 22), pack-reused 1797
Receiving objects: 100% (1870/1870), 295.87 MiB | 20.91 MiB/s, done.
Resolving deltas: 100% (722/722), done.
Updating files: 100% (970/970), done.


In [5]:
os.chdir('./learningbyreading/ext')

In [ ]:
!./install_candc.sh

In [ ]:
# # Gera boxer em xml
# command = "./install_candc.sh"
# subprocess.run(command, shell=True, check=True, capture_output=True)

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}".format(dirbase))
else:
    logging.info("Diretório já existe: {}".format(dirbase))

## Verifica se arquivos estão corretos

In [ ]:
os.chdir('/content/learningbyreading/ext/candc')
!ls


In [ ]:
#!chmod +x bin/candc
!bin/candc --version
!bin/boxer --version

Roda teste

#Importa word2vec

In [25]:
import re
import numpy as np

In [26]:
from nltk.corpus import wordnet as wn
from scipy.spatial.distance import cosine

#Carrega word2vec

In [41]:
from gensim.models import KeyedVectors
import gensim.downloader as api

# Carregar o modelo Word2Vec
word2vec_model = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [42]:
word2vec_model.init_sims(replace=True)

<ipython-input-42-cf6a3503e660>:1: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  word2vec_model.init_sims(replace=True)


#Carrega bert

In [56]:
from transformers import BertTokenizer, BertModel
import torch

# Inicializa o tokenizer e o modelo
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Função para obter embeddings do BERT
def get_bert_embeddings(text):
    # Tokeniza a entrada e converte para tensor
    inputs = tokenizer(text, return_tensors="pt")

    # Obtém os embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Retorna os embeddings da última camada
    return outputs.last_hidden_state

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Exemplo de uso
embeddings = get_bert_embeddings("Hello, world!")
print(embeddings)

#Define função do grafo

In [43]:

class Graph:
    def __init__(self, sentence: str, debug: bool = False):
        self.words = sentence.split(" ")
        self.count = len([word for word in self.words if len(word) > 0])
        self.gLen = self.count
        self.nodes = [None] * (self.count * 2)  # Armazenamento para conceitos
        self.edges = [None] * (self.count * 2)  # Armazenamento para arestas
        self.conceptCount = 0  # Contador para conceitos
        self.edgeCount = 0  # Contador para arestas
        self.repeated = 0
        self.autoInc = 0  # Variável para auto-incremento
        self.debug = debug  # Variável para debug

        if debug:
            print(sentence)

    def get_vector(self, word):
        return word2vec_model[word] if word in word2vec_model else np.zeros(word2vec_model.vector_size)

    # Função para obter embeddings de uma sentença
    def get_sentence_embeddings(sentence):
        # Tokeniza a sentença
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)

        # Obtém os embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # Mapeia tokens para palavras
        token_embeddings = outputs.last_hidden_state.squeeze().numpy()
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

        # Cria um dicionário para mapear palavras aos seus embeddings
        word_embeddings = {}
        current_word = ""
        word_embedding = None

        for i, token in enumerate(tokens):
            # Se é o primeiro subtoken de uma nova palavra
            if token.startswith("##"):
                token = token[2:]  # Remove o prefixo '##'
                current_word += token
                word_embedding += token_embeddings[i]
            else:
                if current_word:
                    # Adiciona a palavra anterior ao dicionário
                    word_embeddings[current_word] = word_embedding / len(current_word.split("##"))
                current_word = token
                word_embedding = token_embeddings[i]

        # Adiciona a última palavra
        if current_word:
            word_embeddings[current_word] = word_embedding / len(current_word.split("##"))

        return word_embeddings

    # Exemplo de uso
    sentence_embeddings = get_sentence_embeddings("Hello, world!")
    for word, embedding in sentence_embeddings.items():
        print(f"Word: {word}, Embedding: {embedding[:5]}...")  # Mostra os primeiros 5 elementos do embedding

    def getCode(self):
        # Implementação do método getCode
        code = f"ma{self.autoInc}"
        self.autoInc += 1
        return code

    def addConcept(self, name: str, code: str, pos: str):
        # Verifica se o conceito já existe e adiciona uma nova instância se necessário
        for i in range(self.conceptCount):
            if self.nodes[i]['code'] == code:
                nCode = self.getCode()
                self.addConcept(name, nCode, pos)
                rName = self.getCode()
                self.addRelationIn("equality", rName, code)
                self.addRelationEx("equality", rName, nCode)
                return

        # Expandir a lista se necessário
        if self.conceptCount >= len(self.nodes):
            self.nodes.append(None)

        # Adicionar o novo conceito
        self.nodes[self.conceptCount] = {'lemma': name, 'code': code, 'pos': pos}
        self.conceptCount += 1

    # def addConcept(self, name: str, code: str, pos: str):
    #     for i in range(self.conceptCount):
    #         if self.nodes[i]['code'] == code:
    #             nCode = self.getCode()
    #             self.addConcept(name, nCode, pos)
    #             rName = self.getCode()
    #             self.addRelationIn("equality", rName, code)
    #             self.addRelationEx("equality", rName, nCode)
    #             return
    #     self.nodes[self.conceptCount] = {'lemma': name, 'code': code, 'pos': pos}
    #     self.conceptCount += 1

    def addRelationIn(self, name: str, code: str, in_relation: str):
        found = False
        for i in range(self.edgeCount):
            if self.edges[i]['code'] == code:
                found = True
                self.edges[i]['in'] = in_relation
                break
        if not found:
            if self.edgeCount >= len(self.edges):
                self.edges.append(None)  # Expande a lista
            self.edges[self.edgeCount] = {'code': code, 'name': name, 'in': in_relation}
            self.edgeCount += 1

    def addRelationEx(self, name: str, code: str, ex: str):
        found = False
        for i in range(self.edgeCount):
            if self.edges[i]['code'] == code:
                found = True
                self.edges[i]['ex'] = ex
                break
        if not found:
            if self.edgeCount >= len(self.edges):
                self.edges.append(None)  # Expande a lista
            self.edges[self.edgeCount] = {'code': code, 'name': name, 'ex': ex}
            self.edgeCount += 1

    # def addRelationIn(self, name: str, code: str, in_relation: str):
    #     found = False
    #     for i in range(self.edgeCount):
    #         if self.edges[i]['code'] == code:
    #             found = True
    #             break
    #     if found:
    #         self.edges[i]['in'] = in_relation
    #     else:
    #         self.edges[self.edgeCount] = {'code': code, 'name': name, 'in': in_relation}
    #         self.edgeCount += 1

    # def addRelationEx(self, name: str, code: str, ex: str):
    #     found = False
    #     for i in range(self.edgeCount):
    #         if self.edges[i]['code'] == code:
    #             found = True
    #             break
    #     if found:
    #         self.edges[i]['ex'] = ex
    #     else:
    #         self.edges[self.edgeCount] = {'code': code, 'name': name, 'ex': ex}
    #         self.edgeCount += 1

    def addIsRelation(self, name: str, code: str):
        found = False
        for i in range(self.edgeCount):
            if self.edges[i]['code'] == code:
                self.edges[i]['name'] = name
                return
        self.edges[self.edgeCount] = {'code': code, 'name': name}
        self.edgeCount += 1

    def getWeight(self, relation: str) -> int:
        if relation == "equality":
            return 1
        elif relation == "is":
            return 4
        elif relation == "rep":
            return 1
        elif relation == "for":
            return 1
        elif relation == "on":
            return 1
        elif relation == "of":
            return 1
        elif relation == "in":
            return 3
        elif relation == "experiencer":
            return 6
        elif relation == "agent":
            return 8
        elif relation == "theme":
            return 8
        else:
            return 1

    def printNodes(self):
        for i in range(self.conceptCount):
            node = self.nodes[i]
            print(f"{node['code']}   {node['lemma']}   {node['pos']}")

    def printEdges(self):
        for i in range(self.edgeCount):
            edge = self.edges[i]
            print(f"{edge['name']}\t{edge.get('in', '')}\t{edge.get('ex', '')}")

    def getNode(self, code: str):
        for i in range(self.conceptCount):
            if self.nodes[i] and self.nodes[i]['code'] == code:
                return self.nodes[i]

        for i in range(self.edgeCount):
            edge = self.edges[i]
            if edge:
                if 'name' in edge and edge['name'] == "equality":
                    if 'ex' in edge and edge['ex'] == code:
                        in_code = edge.get('in')  # Use .get() to avoid KeyError
                        if in_code:
                            return self.getNode(in_code)
        return None  # Return None or a default value if no node is found

    def finalize(self):
        for i in range(self.edgeCount):
            if self.edges[i]['name'] == "is":
                for j in range(self.edgeCount):
                    if self.edges[i]['code'] == self.edges[j]['code']:
                        self.edges[i]['ex'] = self.edges[j]['ex']
                        self.edges[i]['in'] = self.edges[j]['in']
                        print("found")
                        break

    def checkGraph(self) -> bool:
            if self.debug:
                for i in range(self.conceptCount):
                    node = self.nodes[i]
                    if node:
                        print(f"{node['code']}   {node['lemma']}")

            for i in range(self.edgeCount):
                print(i)
                edge = self.edges[i]
                print(edge)
                if edge:
                    in_node_obj = self.getNode(edge.get('in'))
                    ex_node_obj = self.getNode(edge.get('ex'))

                    in_node = in_node_obj['lemma'] if in_node_obj is not None else None
                    ex_node = ex_node_obj['lemma'] if ex_node_obj is not None else None

                    print(f"{in_node}  {edge['name']}   {ex_node}")

                    if edge['name'] == "equality" and in_node == ex_node:
                        self.repeated += 1

                    for j in range(self.edgeCount):
                        other_edge = self.edges[j]
                        if other_edge and edge['name'] == other_edge['name']:
                            in_node_other_obj = self.getNode(other_edge.get('in'))
                            ex_node_other_obj = self.getNode(other_edge.get('ex'))
                            in_node_other = in_node_other_obj['lemma'] if in_node_other_obj is not None else None
                            ex_node_other = ex_node_other_obj['lemma'] if ex_node_other_obj is not None else None

                            if in_node == in_node_other and ex_node == ex_node_other:
                                other_edge['name'] = "rep"
                                self.repeated += 1

            return True if self.repeated > 0 else False

    def matchRelationNameSTS(self, r1, r2):
        print(f"{r1}    {r2}")
        if r1 == "rep" or r2 == "rep":
            return 0.5
        if r1 == "topic" or r2 == "topic":
            return 0.3
        if r1 == "is" and r2 == "is":
            return 0.7
        if r1 == "equality" and r2 == "equality":
            return 0.7
        if r1 == "rep":
            return 0.0
        if r1 == r2:
            return 1
        if (r1 == "experiencer" and r2 == "of") or (r1 == "of" and r2 == "experiencer"):
            return 0.8
        if (r1 == "theme" and r2 == "agent") or (r1 == "agent" and r2 == "theme"):
            return 0.6
        else:
            return 0.79 - 0.01 * (self.getWeight(r1) + self.getWeight(r2))

    # Convertendo a função matchRelationName para Python
    def matchRelationName(self, r1: str, r2: str) -> float:
        if r1 == "is" and r2 == "is":
            return 0.7
        if r1 == "equality" and r2 == "equality":
            return 0.7
        if r1 == "rep":
            return 0.0
        if r1 == r2:
            return 1.0
        else:
            return 0.73


    # Corrigindo a função stringSim uma última vez
    def stringSim(self, s1: str, s2: str) -> float:
        m, n = len(s1), len(s2)

        if m == 0 and n == 0: return 1.0
        if m == 0 or n == 0: return 0.0

        costs = [0] * (n + 1)

        # Initialize the array
        for k in range(n + 1):
            costs[k] = k

        for i, c1 in enumerate(s1):
            costs[0] = i + 1
            corner = i

            for j, c2 in enumerate(s2):
                upper = costs[j + 1]
                if c1 == c2:
                    costs[j + 1] = corner
                else:
                    t = min(upper, corner)
                    costs[j + 1] = min(costs[j], t) + 1
                corner = upper

        result = costs[n]
        return 1 - float(result) / max(m, n)

    def expand(self, word):
      ewords = [word]
      for edge in self.edges:
          if edge and edge['name'] in ["equality", "is", "of"]:
              node_ex = self.getNode(edge['ex'])
              node_in = self.getNode(edge['in'])

              if node_ex and node_ex['lemma'] == word:
                  ewords.append(node_in['lemma']) if node_in else None
              elif node_in and node_in['lemma'] == word:
                  ewords.append(node_ex['lemma']) if node_ex else None

      return ewords

    # def expand(self, word):
    #     ewords = [word]
    #     for edge in self.edges:
    #         if edge and edge['name'] in ["equality", "is", "of"]:
    #             if self.getNode(edge['ex'])['lemma'] == word:
    #                 ewords.append(self.getNode(edge['in'])['lemma'])
    #             elif self.getNode(edge['in'])['lemma'] == word:
    #                 ewords.append(self.getNode(edge['ex'])['lemma'])
    #     return ewords



#Define função similaridade dos grafos

In [44]:
class GraphSTS(Graph):
    def __init__(self, sentence: str, debug: bool = False):
        super().__init__(sentence, debug)

    def matchConcept(self, c1, c2):
        vec1 = self.get_vector(c1)
        vec2 = self.get_vector(c2)
        # Similaridade do cosseno
        return 1 - cosine(vec1, vec2) if np.any(vec1) and np.any(vec2) else 0

    def matchConceptWithExpansion(self, g1, c1, g2, c2):
        all1 = g1.expand(c1)
        all2 = g2.expand(c2)
        print("expansions:", all1, all2)
        max_similarity = 0

        for word1 in all1:
            for word2 in all2:
                similarity = self.matchConcept(word1, word2)
                if similarity > max_similarity:
                    max_similarity = similarity

        return max_similarity

    def matchRelation(self, g1, r1, g2, r2):
        node_g1_in = g1.getNode(r1['in'])
        node_g1_ex = g1.getNode(r1['ex'])
        node_g2_in = g2.getNode(r2['in'])
        node_g2_ex = g2.getNode(r2['ex'])

        # Verifica se algum dos nós é None e retorna 0 em tal caso
        if not all([node_g1_in, node_g1_ex, node_g2_in, node_g2_ex]):
            return 0

        node_in_g1 = node_g1_in['lemma']
        node_ex_g1 = node_g1_ex['lemma']
        node_in_g2 = node_g2_in['lemma']
        node_ex_g2 = node_g2_ex['lemma']

        d1 = self.matchConceptWithExpansion(g1, node_in_g1, g2, node_in_g2)
        d2 = self.matchConceptWithExpansion(g1, node_ex_g1, g2, node_ex_g2)
        d3 = self.matchRelationNameSTS(r1['name'], r2['name'])

        return ((d2 + d1) / 2.0) * d3

    # def matchRelation(self, g1, r1, g2, r2):
    #     node_in_g1 = g1.getNode(r1['in'])['lemma']
    #     node_ex_g1 = g1.getNode(r1['ex'])['lemma']
    #     node_in_g2 = g2.getNode(r2['in'])['lemma']
    #     node_ex_g2 = g2.getNode(r2['ex'])['lemma']
    #     # print('in_nodes:', node_in_g1, node_in_g2)
    #     # print('ex_nodes:', node_ex_g1, node_ex_g2)
    #     # print('relations:', r1['name'], r2['name'])

    #     d1 = self.matchConceptWithExpansion(g1, node_in_g1, g2, node_in_g2)
    #     # print("d1 int:", d1)
    #     d2 = self.matchConceptWithExpansion(g1, node_ex_g1, g2, node_ex_g2)
    #     # print("d2 ext:", d2)
    #     d3 = self.matchRelationNameSTS(r1['name'], r2['name'])  # Correção: removido o 'self' extra
    #     # print("d3 relation:", d3)

    #     return ((d2 + d1) / 2.0) * d3


    # def matchGraph(self, g) -> float:
    #     total_similarity = 0
    #     num_comparisons = 0

    #     for i in range(self.edgeCount):
    #         for j in range(g.edgeCount):
    #             # Obtém os vetores para os conceitos das relações
    #             vec1_in = self.get_vector(self.getNode(self.edges[i]['in'])['lemma'])
    #             vec1_ex = self.get_vector(self.getNode(self.edges[i]['ex'])['lemma'])
    #             vec2_in = g.get_vector(g.getNode(g.edges[j]['in'])['lemma'])
    #             vec2_ex = g.get_vector(g.getNode(g.edges[j]['ex'])['lemma'])

    #             # Calcula a similaridade do cosseno para 'in' e 'ex' e faz a média
    #             similarity_in = 1 - cosine(vec1_in, vec2_in) if np.any(vec1_in) and np.any(vec2_in) else 0
    #             print("vec2_in:", self.getNode(self.edges[i]['in'])['lemma'])
    #             print("vec1_in:", g.getNode(g.edges[j]['in'])['lemma'])
    #             print("sim_in:", similarity_in)
    #             similarity_ex = 1 - cosine(vec1_ex, vec2_ex) if np.any(vec1_ex) and np.any(vec2_ex) else 0
    #             print("vec2_ex:", self.getNode(self.edges[i]['ex'])['lemma'])
    #             print("vec1_ex:", g.getNode(g.edges[j]['ex'])['lemma'])
    #             print("sim_ex:", similarity_in)
    #             average_similarity = (similarity_in + similarity_ex) / 2

    #             total_similarity += average_similarity
    #             num_comparisons += 1

    #     # Calcula a similaridade média entre os grafos
    #     average_graph_similarity = total_similarity / num_comparisons if num_comparisons > 0 else 0

    #     if self.debug:
    #         print(f"Similaridade Média entre os Grafos: {average_graph_similarity}")

    #     return average_graph_similarity

    def calculateGraph(self):
        matrix = np.zeros((self.edgeCount, g.edgeCount))
        for j in range(self.edgeCount):
            max_val = np.max(matrix[:, j])
            if max_val == 0:
                nan_count_d2 += 1
            else:
                rel = self.edges[np.argmax(matrix[:, j])]['name']
                weight = self.getWeight(rel)
                sum_d2 += max_val * weight
                weight_sum_d2 += weight

    def printRelationMatrix(self, matrix, g, export_csv=False, filename="relation_matrix.csv"):
        print("Matriz de Relações:")
        print("     ", end="")
        header = [""] + [g.edges[j]['name'] for j in range(g.edgeCount)]
        print("     ".join(header))

        rows = []
        for i in range(self.edgeCount):
            row = [self.edges[i]['name']]
            print(f"{self.edges[i]['name']:>12}", end="     ")
            for j in range(g.edgeCount):
                print(f"{matrix[i, j]:>12.4f}", end="     ")
                row.append(matrix[i, j])
            rows.append(row)
            print()

        if export_csv:
            with open(filename, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(header)
                csvwriter.writerows(rows)
            print(f"Matriz exportada para {filename}")

    def matchGraph(self, g):
        matrix = np.zeros((self.edgeCount, g.edgeCount))

        for i in range(self.edgeCount):
            for j in range(g.edgeCount):
                similarity = self.matchRelation(self, self.edges[i], g, g.edges[j])
                matrix[i, j] = similarity

        sum_d1, sum_d2, weight_sum_d1, weight_sum_d2 = 0, 0, 0, 0
        nan_count_d1, nan_count_d2 = 0, 0

        for i in range(self.edgeCount):
            max_val = np.max(matrix[i, :])
            if max_val == 0:
                nan_count_d1 += 1
            else:
                rel = g.edges[np.argmax(matrix[i, :])]['name']
                weight = self.getWeight(rel)
                sum_d1 += max_val * weight
                weight_sum_d1 += weight

        for j in range(g.edgeCount):
            max_val = np.max(matrix[:, j])
            if max_val == 0:
                nan_count_d2 += 1
            else:
                rel = self.edges[np.argmax(matrix[:, j])]['name']
                weight = self.getWeight(rel)
                sum_d2 += max_val * weight
                weight_sum_d2 += weight

        d1 = sum_d1 / weight_sum_d1 if weight_sum_d1 > 0 else 0
        d2 = sum_d2 / weight_sum_d2 if weight_sum_d2 > 0 else 0

        self.printRelationMatrix(matrix, g)

        return (d1 + d2) / 2 if self.edgeCount - nan_count_d1 > 0 and g.edgeCount - nan_count_d2 > 0 else 0

#Gera os grafos a partir da classe

In [45]:
def generate_graphs(fres):
    # Regular expressions
    conceptReg = re.compile(r"c\d+:(.+?):.+instance (.+?) ")
    argReg = re.compile(r"c\d+:(.+?):.+arg (.+?) ")
    inRelReg = re.compile(r"(.*?):(.*?):.* int (.*?) ")
    exRelReg = re.compile(r"(.*?):(.*?):.* ext (.*?) ")
    inEqRelReg = re.compile(r"(.*?):equality int (.*?) ")
    exEqRelReg = re.compile(r"(.*?):equality ext (.*?) ")
    isReg = re.compile(r".* (.*?):equality \d+ . is ")
    refReg = re.compile(r".+referent (.+?) .* (.+?) ")

    g1 = GraphSTS(fres.readline(), True)

    for line in fres:
        # Initialize your GraphSTS object here (Placeholder)
        if line and line[0] == '%':
            continue
        else:
            match = conceptReg.search(line)
        if match:
            g1.addConcept(match.group(1), match.group(2), "")
            pass
        elif (match := argReg.search(line)):
            g1.addConcept(match.group(1), match.group(2), "jj")
            pass
        elif (match := inRelReg.search(line)):
            g1.addRelationIn(match.group(2), match.group(1), match.group(3))
            pass
        elif (match := exRelReg.search(line)):
            g1.addRelationEx(match.group(2), match.group(1), match.group(3))
            pass
        elif (match := inEqRelReg.search(line)):
            g1.addRelationIn("equality", match.group(1), match.group(2))
            pass
        elif (match := exEqRelReg.search(line)):
            g1.addRelationEx("equality", match.group(1), match.group(2))
            pass
        elif (match := isReg.search(line)):
            g1.addIsRelation("is", match.group(1))
            pass
        elif (match := refReg.search(line)):
            temp = match.group(2)
            if len(temp) > 3:
                g1.addConcept(match.group(2), match.group(1), "ref")
                pass

    return g1

# def generate_graphs(fres):
#     # Regular expressions
#     conceptReg = re.compile(r"c\d+:(.+?):.+instance (.+?) ")
#     argReg = re.compile(r"c\d+:(.+?):.+arg (.+?) ")
#     inRelReg = re.compile(r"(.*?):(.*?):.* int (.*?) ")
#     exRelReg = re.compile(r"(.*?):(.*?):.* ext (.*?) ")
#     inEqRelReg = re.compile(r"(.*?):equality int (.*?) ")
#     exEqRelReg = re.compile(r"(.*?):equality ext (.*?) ")
#     isReg = re.compile(r".* (.*?):equality \d+ . is ")
#     refReg = re.compile(r".+referent (.+?) .* (.+?) ")

#     g1 = GraphSTS(fres.readline(), True)

#     for line in fres:
#         # Initialize your GraphSTS object here (Placeholder)
#         if line and line[0] == '%':
#             continue
#         else:
#             match = conceptReg.search(line)
#         if match:
#             g1.addConcept(match.group(1), match.group(2), "")
#             pass
#         elif (match := argReg.search(line)):
#             g1.addConcept(match.group(1), match.group(2), "jj")
#             pass
#         elif (match := inRelReg.search(line)):
#             g1.addRelationIn(match.group(2), match.group(1), match.group(3))
#             pass
#         elif (match := exRelReg.search(line)):
#             g1.addRelationEx(match.group(2), match.group(1), match.group(3))
#             pass
#         elif (match := inEqRelReg.search(line)):
#             g1.addRelationIn("equality", match.group(1), match.group(2))
#             pass
#         elif (match := exEqRelReg.search(line)):
#             g1.addRelationEx("equality", match.group(1), match.group(2))
#             pass
#         elif (match := isReg.search(line)):
#             g1.addIsRelation("is", match.group(1))
#             pass
#         elif (match := refReg.search(line)):
#             temp = match.group(2)
#             if len(temp) > 3:
#                 g1.addConcept(match.group(2), match.group(1), "ref")
#                 pass

#     return g1

In [46]:
# import re

# !echo "A mound of something is a large rounded pile of it." | bin/candc --models models/boxer --candc-printer boxer > working/test.ccg
# !echo "A hill is an area of land that is higher than the land that surrounds it." | bin/candc --models models/boxer --candc-printer boxer > working/test2.ccg
# !bin/boxer --input working/test.ccg --output working/drginput.ccg --semantics drg
# !bin/boxer --input working/test2.ccg --output working/drginput2.ccg --semantics drg

# fres = open("working/drginput.ccg", "r")
# fres2 = open("working/drginput2.ccg", "r")
# g1 = generate_graphs(fres)
# g2 = generate_graphs(fres2)


In [47]:
# # g1.checkGraph()
# g2.printNodes()
# print("-------")
# g1.printNodes()

In [48]:
# g1.matchGraph(g2)

#Importa dataset

In [49]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "dataset.csv"

In [50]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia.")

#Processa dataset

In [51]:
import pandas as pd
import os
import subprocess

# Função para processar sentenças e gerar os grafos
def process_sentence(sentence, file_name):
    # Grava a sentença em um arquivo temporário
    with open(file_name, 'w') as file:
        file.write(sentence)

    # Chama o CandC e o Boxer
    subprocess.run(f'echo "{sentence}" | bin/candc --models models/boxer --candc-printer boxer --output /content/CANDC/{file_name}_candc', shell=True)
    subprocess.run(f'bin/boxer --input /content/CANDC/{file_name}_candc --output /content/CANDC/{file_name}_drg --semantics drg', shell=True)



In [52]:
# Lê o arquivo CSV
df = pd.read_csv('/content/CANDC/dataset.csv')
df

,Sentence_Pair,Sentence_1,Sentence_2
0,1.cord:smile,"Cord is strong, thick string.",A smile is the expression that you have on you...
1,2.rooster:voyage,A rooster is an adult male chicken.,A voyage is a long journey on a ship or in a s...
2,3.noon:string,Noon is 12 o'clock in the middle of the day.,"String is thin rope made of twisted threads, u..."
3,4.fruit:furnace,Fruit or a fruit is something which grows on a...,A furnace is a container or enclosed space in ...
4,5.autograph:shore,An autograph is the signature of someone famou...,"The shores or shore of a sea, lake or wide riv..."
...,...,...,...
60,61.cushion:pillow,A cushion is a fabric case filled with soft ma...,A pillow is a rectangular cushion which you re...
61,62.cemetery:graveyard,A cemetery is a place where dead people's bodi...,"A graveyard is an area of land, sometimes near..."
62,63.automobile:car,An automobile is a car.,A car is a motor vehicle with room for a small...
63,64.midday:noon,Midday is 12 o'clock in the middle of the day.,Noon is 12 o'clock in the middle of the day.


In [53]:

# Lista para armazenar os resultados de similaridade
similarities = []

for index, row in df.iterrows():
    # Processa as duas sentenças
    process_sentence(row['Sentence_1'], 'sentence1')
    process_sentence(row['Sentence_2'], 'sentence2')

    # Abre os arquivos processados
    with open("/content/CANDC/sentence1_drg", "r") as fres, open("/content/CANDC/sentence2_drg", "r") as fres2:
        # Gera os grafos
        g1 = generate_graphs(fres)
        g2 = generate_graphs(fres2)

        # Calcula e armazena a similaridade
        similarity = g1.matchGraph(g2)
        similarities.append(similarity)

# Adiciona a coluna de similaridade ao DataFrame
df['Similarity'] = similarities

# Salva o DataFrame modificado como CSV
df.to_csv('/content/CANDC/results.csv', index=False)

A saída de streaming foi truncada nas últimas 5000 linhas.
expansions: ['ancient'] ['regard']
Theme    Theme
expansions: ['times,'] ['regard']
expansions: ['ancient'] ['wise.']
Theme    as
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['sage', 'person']
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['person', 'sage', 'person']
equality    is
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['person', 'sage', 'person']
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['person', 'sage', 'person']
equality    equality
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['wise.']
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['very']
equality    Manner
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['person', 'sage', 'person']
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['regard']
equality    Theme
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['regard']
expansions: ['priest', 'oracle', 'priest', 'priestess'] ['

In [54]:
similarities

[0.11686275401108323,
 0.13347534014532964,
 0.206379621856979,
 0.34123656774673494,
 0.2583547517588983,
 0.08125291401520371,
 0.27548205809029086,
 0.12128937294473871,
 0.12022937693721836,
 0.14091973954190812,
 0.25118177763779054,
 0.38072960029356184,
 0.3334034298658371,
 0.40230303883102236,
 0.4430491452385681,
 0.06861907904098431,
 0.506846186736768,
 0.17297278360774118,
 0.5037417293167201,
 0.3517921177504791,
 0.3116211600576273,
 0.1339854817890695,
 0.2506984162237495,
 0.6110564230754971,
 0.5025487223267555,
 0.27193870715176066,
 0.31673550906280673,
 0.2513781086821109,
 0.19017632369883358,
 0.65836656585998,
 0.2476429099030793,
 0.14114419651031496,
 0.604265002598986,
 0.2592112496495247,
 0.38921619682883224,
 0.5037111718512395,
 0.27169656011206095,
 0.32913436555769293,
 0.6357501845789097,
 0.3698495327391559,
 0.25210932013019915,
 0.26026139342305915,
 0.4553524346500071,
 0.4200241321515219,
 0.3420258388937348,
 0.11970735096239618,
 0.3282279705330

In [55]:
df

,Sentence_Pair,Sentence_1,Sentence_2,Similarity
0,1.cord:smile,"Cord is strong, thick string.",A smile is the expression that you have on you...,0.116863
1,2.rooster:voyage,A rooster is an adult male chicken.,A voyage is a long journey on a ship or in a s...,0.133475
2,3.noon:string,Noon is 12 o'clock in the middle of the day.,"String is thin rope made of twisted threads, u...",0.206380
3,4.fruit:furnace,Fruit or a fruit is something which grows on a...,A furnace is a container or enclosed space in ...,0.341237
4,5.autograph:shore,An autograph is the signature of someone famou...,"The shores or shore of a sea, lake or wide riv...",0.258355
...,...,...,...,...
60,61.cushion:pillow,A cushion is a fabric case filled with soft ma...,A pillow is a rectangular cushion which you re...,0.553688
61,62.cemetery:graveyard,A cemetery is a place where dead people's bodi...,"A graveyard is an area of land, sometimes near...",0.375405
62,63.automobile:car,An automobile is a car.,A car is a motor vehicle with room for a small...,0.351913
63,64.midday:noon,Midday is 12 o'clock in the middle of the day.,Noon is 12 o'clock in the middle of the day.,0.674848
